## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,46.06,71,0,6.91,clear sky
1,1,Esperance,AU,-33.8667,121.9000,73.24,44,100,15.21,overcast clouds
2,2,Butaritari,KI,3.0707,172.7902,81.81,76,1,11.54,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,73.08,78,100,17.69,overcast clouds
4,4,Hobart,AU,-42.8794,147.3294,63.01,57,75,3.00,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Esperance,AU,-33.8667,121.9000,73.24,44,100,15.21,overcast clouds
2,2,Butaritari,KI,3.0707,172.7902,81.81,76,1,11.54,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,73.08,78,100,17.69,overcast clouds
7,7,Georgetown,MY,5.4112,100.3354,79.90,89,20,3.00,few clouds
8,8,Maceio,BR,-9.6658,-35.7353,76.44,94,20,5.75,few clouds
9,9,Vaini,TO,-21.2000,-175.2000,82.56,78,75,10.36,broken clouds
14,14,Balaipungut,ID,1.0500,101.2833,73.81,93,100,2.01,light rain
22,22,Sao Miguel,BR,-5.4667,-35.3667,81.37,77,40,15.52,scattered clouds
28,28,Avarua,CK,-21.2078,-159.7750,77.05,69,40,14.97,scattered clouds
29,29,Ribeira Grande,PT,38.5167,-28.7000,71.20,88,100,7.38,overcast clouds


In [10]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Esperance,AU,73.24,overcast clouds,-33.8667,121.9000,
2,Butaritari,KI,81.81,light rain,3.0707,172.7902,
3,Rikitea,PF,73.08,overcast clouds,-23.1203,-134.9692,
7,Georgetown,MY,79.90,few clouds,5.4112,100.3354,
8,Maceio,BR,76.44,few clouds,-9.6658,-35.7353,
9,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
14,Balaipungut,ID,73.81,light rain,1.0500,101.2833,
22,Sao Miguel,BR,81.37,scattered clouds,-5.4667,-35.3667,
28,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
29,Ribeira Grande,PT,71.20,overcast clouds,38.5167,-28.7000,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Esperance,AU,73.24,overcast clouds,-33.8667,121.9000,Hospitality Esperance
2,Butaritari,KI,81.81,light rain,3.0707,172.7902,Isles Sunset Lodge
3,Rikitea,PF,73.08,overcast clouds,-23.1203,-134.9692,People ThankYou
7,Georgetown,MY,79.90,few clouds,5.4112,100.3354,Cititel Penang
8,Maceio,BR,76.44,few clouds,-9.6658,-35.7353,Poutur Pousada


In [18]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))